<a href="https://colab.research.google.com/github/ipassynk/tennis-email-agent/blob/main/tennis-email-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai
!pip install crewai_tools
!pip install pydantic
!pip install -qU langchain-google-community[gmail]
!pp install google-auth-oauthlib>=1.2.0
!pp install langchain-google-community>=2.0.0
!pp install google-auth>=2.20.0
!pp install google-api-python-client>=2.100.0

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import ScrapeElementFromWebsiteTool
from langchain_google_community import GmailToolkit
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List
import os
import json

In [ ]:
website_url = 'https://torontowinterleague.tenniscores.com/?mod=nndz-TjJiOWtORzkwTlJFb0NVU1NzOD0%3D&team=nndz-WVNXOHhMOD0%3D'
club_name = "Thornhill Park Tennis Club"
date = "11/09"

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
scrape_tool = ScrapeElementFromWebsiteTool(website_url=website_url, css_element=".team_schedule")

scraper_agent = Agent(
    role="Web Scraper",
    goal="Extract specific information from websites",
    backstory="An expert in web scraping who can extract targeted content from web pages.",
    tools=[scrape_tool],
    verbose=True,
    llm="gpt-4o"
)

scrape_task = Task(
    description="Extract row from a schedule table. Use the CSS selector '.team_schedule' to target the table and find row with date = '12/07'.",
    expected_output="A list of the main headlines from CNN.",
    agent=scraper_agent,
)

In [ ]:
class GetCapitanInput(BaseModel):
    team: str = Field(..., description="Name of the team to lookup")

class GetCapitan(BaseTool):
    name: str = "get_capitan"
    description: str = "Fetch capitans emails and names by team name from teams.json"
    args_schema: Type[BaseModel] = GetCapitanInput

    def _run(self, team: str) -> List[dict]:
        try:
            with open("teams.json", "r") as f:
                data = json.load(f)
                capitan = [entry for entry in data if entry['team'].lower() == team.lower()]
                if not capitan:
                    return [{"message": f"No capitans for for team {team}"}]
                return capitan
        except Exception as e:
            return [{"error": str(e)}]

capitan_tool = GetCapitan()

capitan_agent = Agent(
    role="Lookup capitan email and name by team name",
    goal="Find capitan email and name by team name",
    backstory=(
        "You can find opponet capitain email and name by team name. "
    ),
    tools=[capitan_tool],
    verbose=True,
    llm="gpt-4o"
)

capitan_task = Task(
    description="Find capitains emails and name for a team by name",
    expected_output="A list of dictionaries containing the name and email of the team's capitain(s).",
    agent=capitan_agent,
)

In [ ]:
class GetMatchEmailInput(BaseModel):
    date: str = Field(..., description="Match date. For example: '2025-10-27'")
    time: str = Field(..., description="Match time. For example: '7:00pm'")

class GetMatchEmail(BaseTool):
    name:str = "get_match_email"
    description: str = "Create an email subject and body for a match at Thornhill Park Tennis Club. Input: match_date (string), optional time."
    args_schema: Type[BaseModel] = GetMatchEmailInput

    def _run(self, date: str, time: str):
        subject = f"Match {date} {time} at {club_name}"

        body = f"""Hi Ladies,

            Our teams are scheduled to play on {date} at {time} at the {club_name}.
            The club address is 26A Old Yonge St, Thornhill, ON L4J 8C5 (at the corner of Centre St and Yonge St).
            There’s a large parking lot near the clubhouse, but you’ll need to obtain temporary parking permits. Please pick them up from the monitor inside the clubhouse and display them on your dashboards. Additional parking is available just across Old Yonge St near the Thornhill Pub.
            Looking forward to seeing your team tonight!

            Cheers,
            Julia Passynkova
        """
        return {"body": body, "subject": subject}

email_tool = GetMatchEmail()

email_agent = Agent(
    role="Tennis email writter",
    goal="Create draft email",
    backstory=(
        "You can create draft email for oppoonet team having subject and body. "
    ),
    tools=[email_tool],
    verbose=True,
    llm="gpt-4o"
)

email_task = Task(
    description="Create draft reminder email for match.",
    expected_output="Create email body and subject.",
    agent=email_agent,
)

In [ ]:
from langchain_google_community import GmailToolkit
from crewai.tools import BaseTool # Import BaseTool
import pickle
from googleapiclient.discovery import build

def get_gmail_service():
    """Get Gmail service with OAuth credentials."""
    creds = None
    # The file token.pickle stores the user's access and refresh tokens.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    service = build('gmail', 'v1', credentials=creds)
    return service

service = get_gmail_service()
gmail_toolkit = GmailToolkit(api_resource=service)
langchain_gmail_tools = gmail_toolkit.get_tools()

In [ ]:
from langchain_google_community import GmailToolkit
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

service = get_gmail_service()
toolkit = GmailToolkit(api_resource=service)
langchain_gmail_tools = toolkit.get_tools()

class GmailToolWrapper(BaseTool):
    """Wrapper to make LangChain Gmail tools compatible with CrewAI"""
    
    def __init__(self, langchain_tool):
        super().__init__()
        self.langchain_tool = langchain_tool
        self.name = langchain_tool.name
        self.description = langchain_tool.description
        # Copy args_schema if it exists
        if hasattr(langchain_tool, 'args_schema'):
            self.args_schema = langchain_tool.args_schema

    def _run(self, *args, **kwargs):
        return self.langchain_tool.run(*args, **kwargs)

# Create CrewAI compatible tools using the wrapper
crewai_gmail_tools = []
for tool in langchain_gmail_tools:
    crewai_gmail_tools.append(GmailToolWrapper(tool))

print(f"Loaded {len(crewai_gmail_tools)} Gmail tools:")
for tool in crewai_gmail_tools:
    print(f"- {tool.name}: {tool.description}")

gmail_agent = Agent(
    tools=crewai_gmail_tools, # Use the wrapped tools
    role="Tennis email sender",
    goal="Send draft email via GMail",
    backstory=(
        "You can send draft email for opponent emails so I can review and forward the email myself. "
    ),
    verbose=True,
    llm="gpt-4o"
)

gmail_task = Task(
    description=(
        "After email content has been created, use the gmail tools to: "
        "1. Create a draft email with the subject and body from the previous step "
        "2. Set the recipient to the opponent team's captain email "
        "3. Confirm the draft was created successfully"
    ),
    expected_output="A draft email confirming match details.",
    agent=gmail_agent,
)

In [ ]:
supervisor_agent = Agent(
    role="Supervisor Agent",
    goal="Coordinate tasks among team lookup and email agents to prepare match communications.",
    backstory="Understands how to use other agents to accomplish composite goals.",
    tools=[scrape_tool, capitan_tool, email_tool] + crewai_gmail_tools,
    reasoning=True,
    max_reasoning_attempts=3,
    verbose=True,
    llm="gpt-4o"
)

supervisor_task = Task(
    description=(
        "Plan and coordinate the process: "
        "1. Find if Thornhill Park team plays at home (H) for a specific date. "
        "2. If Thornhill Park team plays away game (A) tell this and stop the flow. "
        "3. Get opponent team name. "
        "4. Find opponent email and name. "
        "5. Create email body and subject. "
        "6. CRITICAL: Use the gmail_create_draft tool to create a draft email with the created content and send it to the opponent's email address. "
        "7. Confirm the draft email was created successfully. "
        "Do not mark this task as complete until step 6 and 7 are finished."
    ),
    expected_output="Confirmation of whether the Thornhill Park team plays at home or away on the specified date, and if playing at home, send drafted email via gmail for the opponent team's captain(s).",
    agent=supervisor_agent,
    dependencies=[scrape_task, capitan_task, email_task, gmail_task]
)

In [ ]:
crew = Crew(
  agents=[supervisor_agent],
  tasks=[supervisor_task],
  verbose=True,
  process=Process.sequential,
  color=False
  #inject_date=True,  # Automatically inject current date into tasks
  #date_format="%B %d, %Y",  # Format as "May 21, 2025"
)

In [ ]:
inputs = {
    "date": date,
}

result = crew.kickoff(inputs=inputs)

In [ ]:
# Debug: Check if Gmail tools are properly loaded
print("Available Gmail tools:")
for tool in crewai_gmail_tools:
    print(f"- {tool.name}: {tool.description}")
    
print(f"\nTotal Gmail tools loaded: {len(crewai_gmail_tools)}")

# Test if tools are accessible to the supervisor agent
print(f"\nSupervisor agent tools count: {len(supervisor_agent.tools)}")
print("Supervisor agent tool names:")
for tool in supervisor_agent.tools:
    print(f"- {tool.name}")

# Check if Gmail service is working
try:
    service = get_gmail_service()
    print(f"\nGmail service status: Connected successfully")
except Exception as e:
    print(f"\nGmail service error: {e}")
